Trong quá trình chạy các cell `!pip install ...` bên dưới, Colab có thể hiển thị một thông báo yêu cầu **"RESTART SESSION"**.

**--> VUI LÒNG BẤM NÚT `CANCEL` HOẶC BỎ QUA THÔNG BÁO NÀY.**

In [1]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [38]:
import os

# THAY ĐỔI ĐƯỜNG DẪN DỮ LIỆU ĐẦU VÀO TẠI ĐÂY
# Anh em chịu khó list từng video ra do sợ colab dừng giữa chừng nên tui muốn cho chạy từng video ròi lưu kết quả lại trước khi chạy tiếp video tiếp theo
data_dirs = [
    # '/content/drive/Shareddrives/keyframe_btc/Keyframes_L25/keyframes/L25_V031',
    # '/content/drive/Shareddrives/keyframe_btc/Keyframes_L25/keyframes/L25_V032',
    '/content/drive/Shareddrives/keyframe_btc/Keyframes_L25/keyframes/L25_V033',
    '/content/drive/Shareddrives/keyframe_btc/Keyframes_L25/keyframes/L25_V034',
    '/content/drive/Shareddrives/keyframe_btc/Keyframes_L25/keyframes/L25_V035',
    '/content/drive/Shareddrives/keyframe_btc/Keyframes_L25/keyframes/L25_V036',
    '/content/drive/Shareddrives/keyframe_btc/Keyframes_L25/keyframes/L25_V037',
    '/content/drive/Shareddrives/keyframe_btc/Keyframes_L25/keyframes/L25_V038',
    '/content/drive/Shareddrives/keyframe_btc/Keyframes_L25/keyframes/L25_V039',
    '/content/drive/Shareddrives/keyframe_btc/Keyframes_L25/keyframes/L25_V040',
    '/content/drive/Shareddrives/keyframe_btc/Keyframes_L25/keyframes/L25_V041',
    '/content/drive/Shareddrives/keyframe_btc/Keyframes_L25/keyframes/L25_V042',
    '/content/drive/Shareddrives/keyframe_btc/Keyframes_L25/keyframes/L25_V043',
    '/content/drive/Shareddrives/keyframe_btc/Keyframes_L25/keyframes/L25_V044',
    '/content/drive/Shareddrives/keyframe_btc/Keyframes_L25/keyframes/L25_V045',
    '/content/drive/Shareddrives/keyframe_btc/Keyframes_L25/keyframes/L25_V046',
    '/content/drive/Shareddrives/keyframe_btc/Keyframes_L25/keyframes/L25_V047',
    '/content/drive/Shareddrives/keyframe_btc/Keyframes_L25/keyframes/L25_V048',
    '/content/drive/Shareddrives/keyframe_btc/Keyframes_L25/keyframes/L25_V049',
    '/content/drive/Shareddrives/keyframe_btc/Keyframes_L25/keyframes/L25_V050'
]

# 2. Tiền tố đường dẫn sẽ bị loại bỏ khi lưu kết quả
PATH_PREFIX_TO_REMOVE = '/content/drive/Shareddrives'

# 3. Thư mục gốc để lưu các file parquet kết quả
BASE_OUTPUT_DIR = '/content/drive/Shareddrives/keyframe_btc/processed_ocr_results'

In [30]:
!pip install vietocr --quiet
!pip install opencv-python --quiet
!pip install hezar[vision] --quiet
!pip install --upgrade pillow --quiet
!pip install pyvi==0.1.1 --quiet

ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
vietocr 0.3.13 requires pillow==10.2.0, but you have pillow 10.0.0 which is incompatible.
scikit-image 0.25.2 requires pillow>=10.1, but you have pillow 10.0.0 which is incompatible.
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
vietocr 0.3.13 requires pillow==10.2.0, but you have pillow 11.3.0 which is incompatible.


## Text Detection

In [31]:
from hezar.models import Model
from hezar.utils import load_image, draw_boxes, show_image
import os
from PIL import Image
import numpy as np

# Load the CRAFT text detection model
model = Model.load("hezarai/CRAFT", device="cuda")

In [32]:
from vietocr.tool.predictor import Predictor
from vietocr.tool.config import Cfg

# Define the configuration for the VietOCR model
config = Cfg.load_config_from_name('vgg_transformer')

# Load the pre-trained VietOCR model
config['weights'] = '/content/drive/Shareddrives/workspace/notebook/vgg_transformer.pth'
config['cnn']['pretrained']=False
config['device'] = 'cuda' # Use 'cuda' if you have a GPU, 'cpu' otherwise
detector = Predictor(config)

/usr/local/lib/python3.12/dist-packages/torch/nn/modules/transformer.py:392: UserWarning: enable_nested_tensor is True, but self.use_nested_tensor is False because encoder_layer.self_attn.batch_first was not True(use batch_first for better inference performance)
  warnings.warn(


In [33]:
def group_boxes_into_lines(bboxes, y_threshold=0.7, x_threshold=1.5):
    """
    Groups word-level bounding boxes into line-level bounding boxes.
    A box is added to the current line if its vertical center is close
    to the line's center and it's horizontally close.

    Args:
        bboxes (list): A list of bounding boxes from CRAFT in [x, y, w, h] format.
        y_threshold (float): Percentage of box height for vertical merging.
        x_threshold (float): Multiplier of average box height for horizontal gap.

    Returns:
        list: A list of merged line-level bounding boxes in [x, y, w, h] format.
    """
    if not bboxes:
        return []

    # Sort boxes from top-to-bottom, then left-to-right
    bboxes.sort(key=lambda bbox: (bbox[1], bbox[0]))

    lines = []
    current_line = bboxes[0]

    for bbox in bboxes[1:]:
        # Get properties of the current line and the new box
        line_x, line_y, line_w, line_h = current_line
        box_x, box_y, box_w, box_h = bbox

        # Calculate vertical centers
        line_v_center = line_y + line_h / 2
        box_v_center = box_y + box_h / 2

        # Calculate horizontal distance (gap)
        horizontal_gap = box_x - (line_x + line_w)

        # Condition 1: Check if vertical centers are aligned
        is_vertically_aligned = abs(line_v_center - box_v_center) < (max(line_h, box_h) * y_threshold)

        # Condition 2: Check if horizontal gap is small enough
        is_horizontally_close = horizontal_gap < (max(line_h, box_h) * x_threshold)

        if is_vertically_aligned and is_horizontally_close:
            # Merge the box into the current line
            new_x = min(line_x, box_x)
            new_y = min(line_y, box_y)
            new_w = max(line_x + line_w, box_x + box_w) - new_x
            new_h = max(line_y + line_h, box_y + box_h) - new_y
            current_line = [new_x, new_y, new_w, new_h]
        else:
            # Finish the current line and start a new one
            lines.append(current_line)
            current_line = bbox

    # Add the last line to the list
    lines.append(current_line)

    return lines

In [34]:
# Text correction
from transformers import pipeline
corrector = pipeline("text2text-generation", model="bmd1905/vietnamese-correction-v2", max_new_tokens=None)
MAX_LENGTH = 512

Device set to use cuda:0


In [35]:
import re
import unicodedata
from typing import List

# --- Các hàm con cho việc xử lý ---

def normalize_unicode(text: str) -> str:
    return unicodedata.normalize("NFC", text)

def normalize_whitespace(text: str) -> str:
    text = text.replace("\xa0", " ")
    return re.sub(r"\s+", " ", text).strip()

# --- Hàm xử lý chính cho SBERT ---

def preprocess_for_sbert(
    text: str,
    lowercase: bool = True,
    apply_corrector=None
) -> str:
    """
    Pipeline chuẩn hóa văn bản nhẹ nhàng, phù hợp cho Sentence-BERT.
    """
    if not isinstance(text, str) or not text.strip():
        return ""

    # (1) Unicode & khoảng trắng
    text = normalize_unicode(text)
    text = normalize_whitespace(text)

    # (2) Chuyển thành chữ thường
    if lowercase:
        text = text.lower()

    return text

print("Đã định nghĩa pipeline xử lý văn bản mới cho SBERT.")

Đã định nghĩa pipeline xử lý văn bản mới cho SBERT.


In [ ]:
import pandas as pd
from PIL import Image
import os
from tqdm.notebook import tqdm

OCR_BATCH_SIZE = 512

# --- THIẾT LẬP CÁC ĐƯỜNG DẪN VÀ BIẾN CỐ ĐỊNH ---
# Tiền tố đường dẫn cần loại bỏ khỏi path lưu trữ
PATH_PREFIX_TO_REMOVE = '/content/drive/Shareddrives'
# Thư mục gốc để lưu các file parquet kết quả
BASE_OUTPUT_DIR = '/content/drive/Shareddrives/keyframe_btc/processed_ocr_results'

# Vòng lặp chính, xử lý từng thư mục video một
for data_dir in data_dirs:
    print(f"\n{'='*20} PROCESSING DIRECTORY: {os.path.basename(data_dir)} {'='*20}")

    # 1. THU THẬP TẤT CẢ ĐƯỜNG DẪN ẢNH TRONG THƯ MỤC HIỆN TẠI
    image_paths_current_dir = []
    for root, dirs, files in os.walk(data_dir):
        for file in files:
            if file.endswith(('.jpg', '.jpeg', '.png')):
                image_paths_current_dir.append(os.path.join(root, file))

    if not image_paths_current_dir:
        print(f"Không tìm thấy ảnh trong '{data_dir}'. Bỏ qua.")
        continue

    # --- Pipeline Phát hiện & OCR Tối ưu ---
    batch_cropped_images = []
    batch_metadata = []

    for image_path in tqdm(image_paths_current_dir, desc="1/3: Detecting text"):
        try:
            pil_image = Image.open(image_path).convert("RGB")
            outputs = model.predict(pil_image)
            word_bboxes = outputs[0]["boxes"]

            if word_bboxes:
                line_bboxes = group_boxes_into_lines(word_bboxes)

                # Định dạng lại đường dẫn gốc theo yêu cầu
                formatted_path = image_path.replace(PATH_PREFIX_TO_REMOVE, '')

                for bbox in line_bboxes:
                    if len(bbox) == 4:
                        x, y, w, h = [int(b) for b in bbox]
                        left, upper, right, lower = x, y, x + w, y + h
                        if right > pil_image.width: right = pil_image.width
                        if lower > pil_image.height: lower = pil_image.height
                        if left < 0: left = 0
                        if upper < 0: upper = 0
                        if right <= left or lower <= upper:
                            continue

                        cropped_img = pil_image.crop((left, upper, right, lower))
                        batch_cropped_images.append(cropped_img)
                        batch_metadata.append({'original_image_path': formatted_path})
        except Exception as e:
            print(f"Lỗi khi xử lý ảnh {image_path}: {e}")

    # --- Nhận dạng và xử lý kết quả ---
    if not batch_cropped_images:
        print("Không phát hiện được vùng văn bản nào. Bỏ qua.")
        continue

    print(f"\nPhát hiện được {len(batch_cropped_images)} vùng văn bản.")
    # print("2/3: Performing batch OCR...")
    # ocr_texts = detector.predict_batch(batch_cropped_images)

    all_ocr_texts = []
    num_images = len(batch_cropped_images)
    # Lặp qua các mini-batch
    for i in tqdm(range(0, num_images, OCR_BATCH_SIZE), desc="2/3: Batch OCR"):
        # Lấy ra một lô ảnh nhỏ
        mini_batch_images = batch_cropped_images[i : i + OCR_BATCH_SIZE]
        # Dự đoán cho lô nhỏ đó
        ocr_results_mini_batch = detector.predict_batch(mini_batch_images)
        # Nối kết quả vào danh sách tổng
        all_ocr_texts.extend(ocr_results_mini_batch)
    print("Batch OCR complete.")

    for metadata, text in zip(batch_metadata, all_ocr_texts):
        metadata['ocr_text'] = text.strip()

    # for metadata, text in zip(batch_metadata, ocr_texts):
    #     metadata['ocr_text'] = text.strip()

    ocr_df = pd.DataFrame(batch_metadata)

    # Loại bỏ các dòng không có text sau khi OCR
    ocr_df = ocr_df[ocr_df['ocr_text'].str.strip() != '']
    if ocr_df.empty:
        print("Không có văn bản hợp lệ sau OCR. Bỏ qua.")
        continue

    # --- Pipeline Tiền xử lý văn bản ---
    print("3/3: Correcting and preprocessing text...")
    texts_to_process = ocr_df['ocr_text'].tolist()
    corrected_results = corrector(texts_to_process, batch_size=32, truncation=True)
    corrected_texts = [result['generated_text'] for result in corrected_results]

    ocr_df['processed_text'] = pd.Series(corrected_texts).apply(preprocess_for_sbert)

    # Loại bỏ các hàng không có text sau khi xử lý
    ocr_df = ocr_df[ocr_df['processed_text'].str.strip() != ''].reset_index(drop=True)

    # Bỏ cột ocr_text thô ban đầu nếu muốn
    ocr_df = ocr_df.drop(columns=['ocr_text'])

    # --- LƯU FILE THEO CẤU TRÚC THƯ MỤC YÊU CẦU ---
    try:
        # Trích xuất tên video, ví dụ: 'L27_V004'
        video_folder_name = os.path.basename(data_dir)

        # Trích xuất thư mục cha, ví dụ: 'L27'
        # Path: .../Keyframes_L27/keyframes/L27_V004 -> dirname 2 lần là .../Keyframes_L27
        parent_dir = os.path.dirname(os.path.dirname(data_dir))
        parent_folder_name = os.path.basename(parent_dir).replace('Keyframes_', '')

        # Tạo thư mục lưu trữ nếu chưa tồn tại, ví dụ: .../processed_ocr_results/L27/
        target_dir = os.path.join(BASE_OUTPUT_DIR, parent_folder_name)
        os.makedirs(target_dir, exist_ok=True)

        # Tạo đường dẫn file cuối cùng
        output_filename = f"{video_folder_name}.parquet"
        df_path = os.path.join(target_dir, output_filename)

        print(f"\nĐang lưu {len(ocr_df)} kết quả vào: {df_path}")
        ocr_df.to_parquet(df_path, index=False)
        print("Lưu thành công!")

    except Exception as e:
        print(f"Lỗi khi lưu file cho {data_dir}: {e}")

print(f"\n{'='*25} QUÁ TRÌNH HOÀN TẤT {'='*25}")


==================== PROCESSING DIRECTORY: L25_V033 ====================


1/3: Detecting text:   0%|          | 0/482 [00:00<?, ?it/s]


Phát hiện được 5433 vùng văn bản.


2/3: Batch OCR:   0%|          | 0/11 [00:00<?, ?it/s]

Batch OCR complete.
3/3: Correcting and preprocessing text...

Đang lưu 5433 kết quả vào: /content/drive/Shareddrives/keyframe_btc/processed_ocr_results/L25/L25_V033.parquet
Lưu thành công!

==================== PROCESSING DIRECTORY: L25_V034 ====================


...

2/3: Batch OCR:   0%|          | 0/4 [00:00<?, ?it/s]

Batch OCR complete.
3/3: Correcting and preprocessing text...

Đang lưu 1805 kết quả vào: /content/drive/Shareddrives/keyframe_btc/processed_ocr_results/L25/L25_V040.parquet
Lưu thành công!

==================== PROCESSING DIRECTORY: L25_V041 ====================


1/3: Detecting text:   0%|          | 0/401 [00:00<?, ?it/s]


Phát hiện được 2652 vùng văn bản.


2/3: Batch OCR:   0%|          | 0/6 [00:00<?, ?it/s]